In [0]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string
import datetime
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import gc

class metrics():            
    def __init__ (self , x , y , wt , x1 , y1 , wt1 , bins = 10 , mode = None):
        self.x = x
        self.y = y 
        self.wt = wt 
        self.wt1 = wt1 
        self.x1 =  x1 
        self.y1 = y1 
        self.bins = bins 
        self.mode = mode
        self.dev_ks = None 
        self.oot_ks = None 
        self.res_comp = None 

    def KS_train(self , y_train,y_train_rf,wt,bins = 20 , mode = None):
        pred=pd.DataFrame(y_train)
        pred['Prob_bad']=y_train_rf
        data=pred
        data.rename(columns={'overall_bad': 'bad'}, inplace=True)
        data.columns = ['bad','Prob_bad']
        data['wt'] = wt
        data['good'] = 1 - data.bad
        if mode == 'score':
            data=(data.sort_values(by=(['Prob_bad','bad']),ascending=[True,False])).reset_index(drop=True)
        else :
            data=(data.sort_values(by=(['Prob_bad','bad']),ascending=[False,False])).reset_index(drop=True)
        data['wt_cumsum'] = data['wt'].cumsum()
        data['wt_good'] = data.apply(lambda x: 0 if x['bad'] == 1 else x['wt'],axis=1)
        data['wt_bad'] = data.apply(lambda x: 0 if x['good'] == 1 else x['wt'],axis=1)
        data['bucket'] = pd.cut(data.wt_cumsum, bins)
        grouped = data.groupby('bucket', as_index = False)
        agg1 = grouped.min().Prob_bad
        agg1 = pd.DataFrame(grouped.min().Prob_bad, columns = ['Decile'])
        agg1['Decile']  = pd.Series(list(range(5,105,int(100/bins))))
        agg1['min_scr']= grouped.min().Prob_bad
        agg1['max_scr'] = grouped.max().Prob_bad
        agg1['bads'] = grouped.sum().wt_bad
        agg1['goods'] = grouped.sum().wt_good
        agg1['total'] = agg1.bads + agg1.goods
        agg1['bad_rate'] = (agg1.bads / agg1.total).apply('{0:.2%}'.format)
        agg1['cumm_bad'] =(agg1.bads / data.wt_bad.sum()).cumsum()* 100
        agg1['cumm_good'] =(agg1.goods / data.wt_good.sum()).cumsum()* 100
        agg1['KS']=np.abs(np.round((agg1['cumm_bad']-agg1['cumm_good']),4)) 
        flag = lambda x: '<----' if x == agg1.KS.max() else ''
        agg1['max_ks'] = agg1.KS.apply(flag)
        auc = roc_auc_score(y_train,y_train_rf,sample_weight = wt )
        gini = 2*auc - 1
        agg1['lcum_bad'] = agg1['cumm_bad'].shift(1)
        agg1['lcum_bad'].fillna(0.0 , inplace = True)
        agg1['cum_total'] = ((agg1['total']/agg1['total'].sum()) * 100).cumsum()
        agg1['lcum_total'] = agg1['cum_total'].shift(1)
        agg1['lcum_total'].fillna(0.0 , inplace = True)
        agg1['area'] = (0.5*((agg1['cumm_bad'] - (agg1['cum_total'])) + (agg1['lcum_bad'] - (agg1['lcum_total']))) * ((agg1['cum_total']) - (agg1['lcum_total'])))
        agg1['cum_area'] = agg1['area'].cumsum()
        agg1['areab'] =  0.5 - ((1*(agg1['bads'].sum()/agg1['total'].sum())/2))
        agg1['PI'] = agg1['cum_area'] / agg1['areab']
        gini = 2*auc - 1
        agg1['gini'] = gini
        agg1['auc'] = auc
        agg1['PI'] = agg1['PI']/10000
        return agg1
    
    def result_ks(self):
        bins = self.bins 
        mode = self.mode 
        xd = self.x
        yd = self.y 
        xo = self.x1 
        yo = self.y1 
        wtd = self.wt
        wto  = self.wt1
        a1 = self.KS_train(xd ,  yd , wtd  , bins , mode)
        a2 = self.KS_train(xo , yo , wto , bins , mode)
        self.dev_ks = a1 
        self.oot_ks = a2 
        res = pd.DataFrame({},index=list(range(0,5)),columns=['Development','Out_of_time'])
        res['Development'][0] = a1.auc[0]
        res['Development'][1] = a1.gini[0]
        res['Development'][2] = a1.KS.max()
        res['Development'][3] = a1.query('Decile == 5')['cumm_bad'].reset_index(drop = True)[0]
        res['Development'][4] = a1.query('Decile == 15')['cumm_bad'].reset_index(drop = True)[0]
        res['Out_of_time'][0] = a2.auc[0]
        res['Out_of_time'][1] = a2.gini[0]
        res['Out_of_time'][2] = a2.KS.max()
        res['Out_of_time'][3] = a2.query('Decile == 5')['cumm_bad'].reset_index(drop = True)[0]
        res['Out_of_time'][4] = a2.query('Decile == 15')['cumm_bad'].reset_index(drop = True)[0]
        res.index = [['ROC_AUC','Gini','KS','10%','20%']]
        self.res_comp = res , a1 , a2 